<a id='table_of_contents'></a>

0. [Import libraries](#imports)
1. [Import data](#import_data)
2. [Data exploration](#data_exploration)
3. [Data cleaning](#data_cleaning)
4. [Data preparation](#data_preparation)
5. [Benchmark model](#benchmark_model)<br>
6. [Feature engineering](#feature_engineering)
7. [Model training](#model_training)<br>
    7.1. [Decision Tree](#decision_tree)<br>
    7.2. [Random Forest](#random_forest)<br>
    7.3. [Comparing algorithm performances](#comparing_algorithm_performances)
8. [Tuning](#tuning)<br>

# 0. Import libraries <a id='imports'></a>
[Back to top](#table_of_contents)

In [9]:
import pandas as pd
import os 
import glob 

# 1. Import data <a id='import_data'></a>
[Back to top](#table_of_contents)

In [12]:
# use glob to get all the parquet files  
# in the folder 
path = "./data/"
pqt_files = glob.glob(os.path.join(path, "*.parquet")) 

In [13]:
pqt_files

['./data\\yellow_tripdata_2024-01.parquet',
 './data\\yellow_tripdata_2024-02.parquet',
 './data\\yellow_tripdata_2024-03.parquet',
 './data\\yellow_tripdata_2024-04.parquet',
 './data\\yellow_tripdata_2024-05.parquet',
 './data\\yellow_tripdata_2024-06.parquet']

In [17]:
dfs = []

In [22]:
df_taxi = pd.concat((pd.read_parquet(f) for f in pqt_files), ignore_index=True)  

In [27]:
df_taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [28]:
df_taxi.shape

(20332093, 19)

In [29]:
df_taxi.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
count,2.033209e+07,1.835611e+07,2.033209e+07,1.835611e+07,2.033209e+07,2.033209e+07,2.033209e+07,2.033209e+07,2.033209e+07,2.033209e+07,2.033209e+07,2.033209e+07,2.033209e+07,2.033209e+07,1.835611e+07,1.835611e+07
mean,1.758162e+00,1.332897e+00,4.704865e+00,2.226112e+00,1.647972e+02,1.638971e+02,1.102157e+00,1.888715e+01,1.408668e+00,4.824373e-01,3.270078e+00,5.468174e-01,9.707874e-01,2.743595e+01,2.249696e+00,1.434393e-01
std,4.296279e-01,8.304837e-01,3.592496e+02,1.053827e+01,6.434706e+01,6.954032e+01,6.315719e-01,7.728530e+01,1.819553e+00,1.213081e-01,4.008415e+00,2.214600e+00,2.315571e-01,7.855135e+01,8.404556e-01,4.933100e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.285400e+03,-7.500000e+00,-5.000000e-01,-3.000000e+02,-1.172600e+02,-1.000000e+00,-1.315970e+03,-2.500000e+00,-1.750000e+00
25%,2.000000e+00,1.000000e+00,1.020000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,9.300000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.564000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.750000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.350000e+01,1.000000e+00,5.000000e-01,2.600000e+00,0.000000e+00,1.000000e+00,2.064000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,3.300000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.190000e+01,2.500000e+00,5.000000e-01,4.160000e+00,0.000000e+00,1.000000e+00,2.985000e+01,2.500000e+00,0.000000e+00
max,6.000000e+00,9.000000e+00,3.127223e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,3.340763e+05,6.599000e+01,3.584000e+01,9.999900e+02,1.702880e+03,1.000000e+00,3.341453e+05,2.520000e+00,1.750000e+00


# 2. Data exploration <a id='data_exploration'></a>
[Back to top](#table_of_contents)

# 3. Data cleaning <a id='data_cleaning'></a>
[Back to top](#table_of_contents)

# 4. Data preparation <a id='data_preparation'></a>
[Back to top](#table_of_contents)

# 5. Benchmark model <a id='benchmark_model'></a>
[Back to top](#table_of_contents)

# 6. Feature engineering <a id='feature_engineering'></a>
[Back to top](#table_of_contents)

# 7. Model training <a id='model_training'></a>
[Back to top](#table_of_contents)

## 7.1. Decision tree <a id='decision_tree'></a>

## 7.2. Random forest <a id='random_forest'></a>

## 7.3. Comparing algorithm performances <a id='comparing_algorithm_performances'></a>

# 8. Tuning <a id='tuning'></a>
[Back to top](#table_of_contents)